https://www.kaggle.com/ateplyuk/pytorch-efficientnet

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.image as mpimg

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils 
from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
train_dir = './train/'
test_dir = './test/'

In [3]:
labels = pd.read_csv("train.csv")
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [4]:
class ImageData(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name = self.df.id[index]
        label = self.df.has_cactus[index]
        
        img_path = os.path.join(self.data_dir, img_name)
        image = mpimg.imread(img_path)
        image = self.transform(image)
        return image, label

In [5]:
data_transf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

train_data = ImageData(df=labels, data_dir = train_dir, transform=data_transf)
train_loader = DataLoader(dataset = train_data, batch_size=64)

In [6]:
!pip install efficientnet_pytorch

  Stored in directory: /Users/dongheelee/Library/Caches/pip/wheels/27/56/13/5bdaa98ca8bd7d5da65cc741987dd14391b87fa1a09081d17a
Successfully built efficientnet-pytorch


In [7]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b1')

In [8]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [10]:
# Unfreeze model weights (불러온 모형의 파라미터를 트레이닝 가능하게 만듬)
for param in model.parameters():
    param.require_grad = True

In [12]:
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

In [18]:
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    model = model.to("cuda")

In [21]:
optimizer = optim.Adam(model.parameters())
loss_func = nn.BCELoss()

In [22]:
# Train model
loss_log = []

for epoch in range(5):
    model.train()
    for ii, (data, target) in enumerate(train_loader):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        target = target.float()
        
        optimizer.zero_grad()
        output = model(data)
        m = nn.Sigmoid()
        loss = loss_func(m(output), target)
        loss.backward()
        optimizer.step()
        
        if ii % 1000 == 0:
            loss_log.append(loss.item())
            
    print("Epoch : {} - Loss : {:.6f}".format(epoch+1, loss.item()))

/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([28])) that is different to the input size (torch.Size([28, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch : 1 - Loss : 0.300911
Epoch : 2 - Loss : 0.269877


KeyboardInterrupt: 

In [23]:
submit = pd.read_csv("sample_submission.csv")
test_data = ImageData(df=submit, data_dir=test_dir, transform=data_transf)
test_loader = DataLoader(dataset=test_data, shuffle=False)

In [25]:
predict = []

model.eval()
for i, (data, _) in enumerate(test_loader):
#     data = data.cuda()
    output = model(data)
    
    pred = torch.sigmoid(output)
    predicted_vals = pred>0.5
    predict.append(int(predicted_vals))
    
submit['has_cactus'] = predict

KeyboardInterrupt: 